# Тьюнинг EllipticEnvelope

## Field `Value`

In [1]:
import numpy  as np
import pandas as pd

In [2]:
df_trn = pd.read_csv('../data/training_le.csv')
df_tst = pd.read_csv('../data/test_le.csv')

In [3]:
X_trn = df_trn.drop(columns=['FraudResult'], axis=1)
y_trn = df_trn['FraudResult']

X_tst = df_tst

**Frauds**

In [4]:
from collections import Counter
print('Train Label Distribution: {}'.format(Counter(y_trn)))

Train Label Distribution: Counter({0: 95469, 1: 193})


In [5]:
num_transactions = df_trn.shape[0]
num_otliers = df_trn['FraudResult'].value_counts()[1]
num_inliers = df_trn['FraudResult'].value_counts()[0]
outliers_fraction = num_otliers / num_transactions
print('Train outliers fraction:', round(outliers_fraction, 3))

Train outliers fraction: 0.002


In [6]:
print('Number of outliers on union dataset =', round((df_trn.shape[0]+df_tst.shape[0])*outliers_fraction, 0))

Number of outliers on union dataset = 284.0


**Union dataframes**

In [7]:
columns4drop = [
    'BatchId',
    'SubscriptionId',
    'CustomerId',
    'TransactionStartTime'
]

In [8]:
df_trn_cut = df_trn.drop(columns=columns4drop, axis=1)
df_tst_cut = df_tst.drop(columns=columns4drop, axis=1)

df_trn_cut = df_trn_cut.drop(columns='FraudResult', axis=1)

print(df_trn_cut.shape, df_tst_cut.shape)

(95662, 8) (45019, 8)


In [9]:
frames = [df_trn_cut, df_tst_cut]
df_union = pd.concat(frames)
df_union.shape

(140681, 8)

In [10]:
df_union.head()

,AccountId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,PricingStrategy
0,3956,5,9,0,2,1000.0,1000,2
1,4840,3,5,2,1,-20.0,20,2
2,4228,5,0,0,2,500.0,500,2
3,647,0,20,9,2,20000.0,21800,2
4,4840,3,5,2,1,-644.0,644,2


## Only `Value`

In [11]:
columns4drop = [
    'AccountId',
    'ProviderId',
    'ProductId',
    'ProductCategory',
    'ChannelId',
    'Amount',
    'PricingStrategy',
]

In [12]:
df_value = df_union.drop(columns=columns4drop, axis=1)

In [13]:
df_value.head()

,Value
0,1000
1,20
2,500
3,21800
4,644


## Running

In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [15]:
from sklearn.covariance import EllipticEnvelope

In [16]:
def running_EE(X, y_eta_trn):
    print(' '*37, 'Confusion-Matrix')
    print('Algorithm', ' '*30, 'TN       FP')
    print(' '*17, 'Frd_tst  Frd_trn       FN       TP     Precision  Recall     F1-score')
    print('='*100)    

    algorithm = EllipticEnvelope(contamination=outliers_fraction)
    y_pred = algorithm.fit(X).predict(X)

    y_pred = [ 1 if x == -1 else 0 for x in y_pred ]
    y_pred_trn = y_pred[ : len(y_trn)]
    y_pred_tst = y_pred[len(y_trn) : ]

    n_frauds_trn      = Counter(y_pred_trn)[1]
    n_frauds_tst      = Counter(y_pred_tst)[1]
    tn, fp, fn, tp    = confusion_matrix(y_eta_trn, y_pred_trn).ravel()
    precision         = precision_score(y_eta_trn, y_pred_trn)
    recall            = recall_score(y_eta_trn, y_pred_trn)
    f1                = f1_score(y_eta_trn, y_pred_trn)

    print('%s %8i %8i' % (' '*34, tn, fp))
    print('%s %8i %8i %8i %8i %10.4f %10.4f %10.4f' % 
          ('EE    '+' '*10, n_frauds_tst, n_frauds_trn, fn, tp, precision, recall, f1))
        
    print('-'*100)
    return

In [17]:
running_EE(df_value, y_trn)

                                      Confusion-Matrix
Algorithm                                TN       FP
                  Frd_tst  Frd_trn       FN       TP     Precision  Recall     F1-score
                                      95411       58
EE                     55      181       70      123     0.6796     0.6373     0.6578
----------------------------------------------------------------------------------------------------


## Choosing Scaler

In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

In [22]:
def running_EE_scaler(name, X, y_eta_trn):
    algorithm = EllipticEnvelope(contamination=outliers_fraction, support_fraction=1.0)
    y_pred = algorithm.fit(X).predict(X)

    y_pred = [ 1 if x == -1 else 0 for x in y_pred ]
    y_pred_trn = y_pred[ : len(y_trn)]
    y_pred_tst = y_pred[len(y_trn) : ]

    n_frauds_trn      = Counter(y_pred_trn)[1]
    n_frauds_tst      = Counter(y_pred_tst)[1]
    tn, fp, fn, tp    = confusion_matrix(y_eta_trn, y_pred_trn).ravel()
    precision         = precision_score(y_eta_trn, y_pred_trn)
    recall            = recall_score(y_eta_trn, y_pred_trn)
    f1                = f1_score(y_eta_trn, y_pred_trn)

    print('%s %8i %8i' % (' '*56, tn, fp))
    print('%s %8i %8i %8i %8i %10.4f %10.4f %10.4f' % 
          (name, n_frauds_tst, n_frauds_trn, fn, tp, precision, recall, f1))
        
    return

In [31]:
print(' '*59, 'Confusion-Matrix')
print('Scaler', ' '*55, 'TN       FP')
print(' '*39, 'Frd_tst  Frd_trn       FN       TP     Precision  Recall     F1-score')
print('='*110)    

X = df_value
running_EE_scaler('Unscaled data                         ', X, y_trn)
running_EE_scaler('Standard scaling                      ', StandardScaler().fit_transform(X), y_trn)
running_EE_scaler('Min-max scaling                       ', MinMaxScaler().fit_transform(X), y_trn)
running_EE_scaler('Max-abs scaling                       ', MaxAbsScaler().fit_transform(X), y_trn)
running_EE_scaler('Robust scaling (without params)       ', RobustScaler().fit_transform(X), y_trn)
running_EE_scaler('Robust scaling (quantile=(25, 75))    ', RobustScaler(quantile_range=(25, 75)).fit_transform(X), y_trn)
running_EE_scaler('Power transformation (Yeo-Johnson)    ', PowerTransformer(method='yeo-johnson').fit_transform(X), y_trn)
running_EE_scaler('Power transformation (Box-Cox)        ', PowerTransformer(method='box-cox').fit_transform(X), y_trn)
running_EE_scaler('Quantile transformation (gaussian pdf)', QuantileTransformer(output_distribution='normal').fit_transform(X), y_trn)
running_EE_scaler('Quantile transformation (uniform pdf) ', QuantileTransformer(output_distribution='uniform').fit_transform(X), y_trn)
# running_EE_scaler('Sample-wise L2 normalizing            ', Normalizer().fit_transform(X), y_trn)

print('-'*110)

                                                            Confusion-Matrix
Scaler                                                         TN       FP
                                        Frd_tst  Frd_trn       FN       TP     Precision  Recall     F1-score
                                                            95411       58
Unscaled data                                55      181       70      123     0.6796     0.6373     0.6578
                                                            95411       58
Standard scaling                             55      181       70      123     0.6796     0.6373     0.6578
                                                            95411       58
Min-max scaling                              55      181       70      123     0.6796     0.6373     0.6578
                                                            95410       59
Max-abs scaling                              94      231       21      172     0.7446     0.8912     0.8113
      

## Max-abs Scaling

In [36]:
def scaleColumns(data, cols_to_scale, scaler):
    for col in cols_to_scale:
        data[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data[col])))
    return data

In [57]:
df_value_max = scaleColumns(df_value, ['Value'], MaxAbsScaler())

In [58]:
df_value_max.head()

,Value
0,0.000101
1,0.000002
2,0.000051
3,0.002206
4,0.000065


In [59]:
df_value_max.shape

(140681, 1)

## Grid Search CV

In [61]:
X1 = df_value_max[ : df_trn.shape[0]]
y1 = y_trn.astype(np.int8)
y1[y_trn==1] = -1
y1[y_trn==0] = 1

In [64]:
from sklearn.model_selection import StratifiedKFold

In [66]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
folds = list(skf.split(X1, y1))

In [67]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

In [68]:
ee = EllipticEnvelope()
params = {
    'support_fraction': np.linspace(0.70, 0.99, num=30),
    'contamination'   : np.linspace(0.0015, 0.0025, num=101)
}
f1score = make_scorer(f1_score)
grdsrch = GridSearchCV(estimator=ee, param_grid=params, scoring=f1score, cv=folds, n_jobs=-1)
grdsrch.fit(X1, y1);

In [69]:
df_res = pd.DataFrame(grdsrch.cv_results_)
df_res[['mean_test_score', 'params']].sort_values(by=['mean_test_score'], ascending=False).head()

,mean_test_score,params
2652,0.999581,"{'contamination': 0.00238, 'support_fraction':..."
2651,0.999581,"{'contamination': 0.00238, 'support_fraction':..."
2649,0.999581,"{'contamination': 0.00238, 'support_fraction':..."
2619,0.999581,"{'contamination': 0.00237, 'support_fraction':..."
1539,0.999581,"{'contamination': 0.00201, 'support_fraction':..."


In [70]:
grdsrch.best_estimator_

EllipticEnvelope(assume_centered=False, contamination=0.00201,
                 random_state=None, store_precision=True,
                 support_fraction=0.79)

**Make predictions with our optimized envelope fit**

In [71]:
ee_opt = grdsrch.best_estimator_
X_tst = df_value[df_trn.shape[0] : ]
y_pred = ee_opt.predict(X_tst)
y_pred = [ 1 if x == -1 else 0 for x in y_pred ]
Counter(y_pred)

Counter({0: 44946, 1: 73})

**Submission**

In [56]:
df_sbm = pd.read_csv('../data/sample_submission.csv')
df_sbm['FraudResult'] = y_pred
df_sbm.to_csv('../submitted/AlBo0720_EE_Value.csv', encoding='utf-8', index=False)

**Result:** `0.0`

### Dummy submitted

In [73]:
df_tst = pd.read_csv('../data/test_le.csv')

In [77]:
columns4drop = [
    'BatchId',
    'SubscriptionId',
    'CustomerId',
    'AccountId',
    'ProviderId',
    'ProductId',
    'ProductCategory',
    'ChannelId',
    'Amount',
    'TransactionStartTime',
    'PricingStrategy',
]

In [78]:
df_tst_cut = df_tst.drop(columns=columns4drop, axis=1)

In [79]:
df_tst_cut.head()

,Value
0,1000
1,2000
2,50
3,3000
4,60


In [81]:
X_tst = df_tst_cut
y_pred = EllipticEnvelope(contamination=outliers_fraction).fit(X_tst).predict(X_tst)
y_pred = [ 1 if x == -1 else 0 for x in y_pred ]
Counter(y_pred)

Counter({0: 44964, 1: 55})

In [82]:
df_sbm = pd.read_csv('../data/sample_submission.csv')
df_sbm['FraudResult'] = y_pred
df_sbm.to_csv('../submitted/AlBo0720_EE_Value.csv', encoding='utf-8', index=False)

**Result:** `0.581818181818182`

max-abs scaler

In [83]:
df_tst_cut_max = scaleColumns(df_tst_cut, ['Value'], MaxAbsScaler())

In [84]:
df_tst_cut_max.head()

,Value
0,0.000105
1,0.000211
2,0.000005
3,0.000316
4,0.000006


In [86]:
X_tst = df_tst_cut_max
y_pred = EllipticEnvelope(contamination=0.00201, support_fraction=0.79).fit(X_tst).predict(X_tst)
y_pred = [ 1 if x == -1 else 0 for x in y_pred ]
Counter(y_pred)

Counter({0: 44925, 1: 94})

In [87]:
df_sbm = pd.read_csv('../data/sample_submission.csv')
df_sbm['FraudResult'] = y_pred
df_sbm.to_csv('../submitted/AlBo0720_EE_Value.csv', encoding='utf-8', index=False)

**Result:** `0.684931506849315`